# Area Analysis Api
## Introduction
This notebook will help you make calls to the TomTom Area Analysis Api. Some of the back-end guts live in the TomTomApi.py module, but hopefully all the adjustments that you want to make can be made here in the notebook. The TomTom Api works by having you create a JSON request object that contains all the details of what information you want -- for example, the area you'd like to cover, the date ranges you're interested in, and which hours of the day you'd like. If you would like hourly buckets then the largest time range you can ask for at once is a day (TomTom only supports 24 time buckets per request). Note that if you make a request for more than a day, TomTom will simply aggregate the days by hour, so that the 07:00 to 08:00 bucket would have traffic data for all the days in the request added together. This isn't usually what we want, so this notebook makes the assumption that you will ask for just one particular date, and will get all 24 hour buckets for that date.

Since it can take TomTom some time (minutes) to prepare the area analysis results, they have broken their api into separate POST and GET calls. First you make a POST call and supply the JSON request object. The POST API will return a jobID and TomTom will get to work on preparing the data. To see how TomTom is doing, and to get the results when they're ready, you must make a GET call with the jobID. The cells in this notebook should help you do this, but this is just a heads up to why there are a couple steps you must take.

When TomTom has finishing preparing the data, it will return a url to a geojson file. You can navigate to this url in your browser and download the file. It will probably be about 20MB in size.

Lastly, since it may be cumbersome to have a separate geojson file for each day of results, there is a utility function at the end of the notebook to combine a collection of geojson files into one file. This is convenient and saves on overall space, since the combined file is smaller than the simple sum of all the constituent files (there are portions of the geojson files that are identical across dates, like the network shape and segment geometries).

## Choose dates and other parameters here

In [3]:
import athena
import utils
from datetime import datetime
import requests
import json

date = "2018-09-23" # Change this and then execute the cell. Must use YYYY-MM-DD format.

# Below here we build the request object. You shouldn't need to adjust anything here.
datetime_object = datetime.strptime(date, "%Y-%m-%d")
day_of_the_week = utils.days[datetime_object.weekday()]
requestObject = {
  "jobName":"DFW " + date, # This name is for our convenience and can be anything
  "distanceUnit":"MILES",
  "network": utils.dfw_network, # Defined in apiHelper.py. It's a rectangle around DFW.
  "dateRange": {
    "name":datetime_object.strftime("%a %b %-d %Y"), # Again this name is just for our convenience and can be anything.
    "from":date,
    "to":date,
  },
  "timeSets": utils.get_24_hours(day_of_the_week) # We want all 24 hour buckets.
}

print("Created request object for " + date + ". Ready to POST a request.")

Created request object for 2018-09-23. Ready to POST a request.


## Execute this cell to send a new request to TomTom
This sends the request to TomTom. It will return a jobId. We will use that jobId to ask TomTom if it is done processing the request in the next cell.

NOTE: There seems to be some problem in verifying the SSL/TLS cert for api.tomtom.com, so I have disabled verification. Because this prints an ugly warning message I capture (hide) everything except stdout with %%capture.

In [7]:
%%capture --no-stdout

API_KEY = athena.tom_tom.get_api_key()
url = "https://api.tomtom.com/traffic/trafficstats/areaanalysis/1?key={}".format(API_KEY) 
response = requests.post(url, verify=False, json=requestObject)
if (response.status_code == 200):
    print("Successful POST")
    job_id = json.loads(response.text)["jobId"]
    print("jobId = " + job_id)
else:
    print("Error in POST")
    print(response.status_code)
    print(response.text)

Successful POST
jobId = 529251


## Execute this cell to check on a request
This cell will make a call to the TomTom API to check on the status of the job `jobId`.

When the job is complete the response will contain urls to geojson, json, and zip files that contain the results. We exclusively use the geojson file. You can manually copy and paste the url into your browser and the file will download.

NOTE: There seems to be some problem in verifying the SSL/TLS cert for api.tomtom.com, so I have disabled verification. Because this prints an ugly warning message I capture (hide) everything except stdout with %%capture.

In [8]:
%%capture --no-stdout
url_get = "https://api.tomtom.com/traffic/trafficstats/status/1/{}?key={}".format(job_id, API_KEY)
response = requests.get(url_get, verify=False)
print(response.status_code)
print(response.text)

200
{"responseStatus":"OK","jobId":"529251","jobState":"DONE","urls":["https://prod-trafficstats-tomtom.s3.eu-west-1.amazonaws.com/2019/09/09/529251/1de677b8/DFW_2018-09-23.geojson","https://prod-trafficstats-tomtom.s3.eu-west-1.amazonaws.com/2019/09/09/529251/1de677b8/DFW_2018-09-23.json","https://prod-trafficstats-tomtom.s3.eu-west-1.amazonaws.com/2019/09/09/529251/1de677b8/DFW_2018-09-23.shapefile.zip"]}
